In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [4]:
csv_file = 'dataset.csv'

# read only 1000 reviews for speed
df = pd.read_csv(csv_file, nrows=1_000, index_col=0)

df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x2909,x2910,x2911,x2912,x2913,x2914,x2915,x2916,y0,y1
0,113904,143004,138473,123201,74545,91365,88969,85869,107115,137517,...,139991,139991,139991,139991,139991,139991,139991,139991,1,0
1,113904,58954,142003,75382,139878,143058,129539,129539,113904,123201,...,139991,139991,139991,139991,139991,139991,139991,139991,1,0
2,113904,132799,64839,143186,107549,58954,142003,137408,57334,105001,...,139991,139991,139991,139991,139991,139991,139991,139991,1,0
3,113904,146172,113152,32380,58954,41362,39803,58954,75382,93190,...,139991,139991,139991,139991,139991,139991,139991,139991,0,1
4,113904,64985,23773,32380,127530,139504,87301,123201,55390,138473,...,139991,139991,139991,139991,139991,139991,139991,139991,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,113904,119684,32710,73083,143058,113904,20453,19951,27544,52382,...,139991,139991,139991,139991,139991,139991,139991,139991,1,0
996,113904,132799,139857,20897,143058,113904,132799,80347,39026,53509,...,139991,139991,139991,139991,139991,139991,139991,139991,0,1
997,113904,132799,47425,42478,57334,81555,28504,127174,52900,102649,...,139991,139991,139991,139991,139991,139991,139991,139991,0,1
998,113904,47019,142017,129370,122226,32710,23569,57334,86715,143186,...,139991,139991,139991,139991,139991,139991,139991,139991,0,1


In [6]:
x = df.iloc[:,:-2]
y = df.iloc[:,-2:]
y

,y0,y1
0,1,0
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
995,1,0
996,0,1
997,0,1
998,0,1


In [ ]:
class MyCustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.Tensor(x)
        self.y = torch.Tensor(y)
        self.n_samples = len(x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

loader = DataLoader(
    x,
    batch_size = 5,
    shuffle = False
)

criterion = nn.MSELoss()